### Adding The Read-Later Feature:

Lets start with adding the button.<br>
1. Navigate to post-detail.html template and add button with a form to it.iter

My post-detail.html template look like the following block:

In [ ]:
{% extends "base.html" %}
{% load static %}

{% block title %}{{ post.title }}{% endblock %}

{% block css_files %}
    <link rel="stylesheet" href="{% static "blog/post-detail.css" %}">
{% endblock %}

{% block content %}
    {% if form.errors %}
        <div class='error-box'>
            <h3>Error</h3>
            <h4>Sorry, we couldn't submit your comment to the server.</h4>
            <p>Please check your inputs and resubmit the form.</p>
        </div>
    {% endif %}

    <section id='summary'>
        <h2>{{ post.title }}</h2>
        <p>
            {% for tag in post_tags %}
                <span class='tag'>{{ tag.caption }} </span>
            {% endfor %}
        </p>
        <article>
            <img src="{{ post.image.url }}" alt="{{ post.title }}">
            <address>By <a href='mailto:{{ post.author.email }}'>{{ post.author }}</a></address>
            <div>
                Last Update on <time>{{ post.date|date:"d M Y" }}</time>
            </div>
        </article>
        <form action="" method="post" class='read-later-form'>
            {% csrf_token %}
            <button type='submit'>Read Later</button>
        </form>
    </section>
    <main>
        <p>
            {{ post.content }}
        </p>
    </main>

    <section class='comments'>
        <ul>
            {% for comment in comments %}
                <li>
                    <h2>{{ comment.user_name }}:</h2>
                    <p>{{ comment.text | linebreaks }}</p>
                </li>
            {% endfor %}
        </ul>
    </section>

    <div class='comments-section'>
        <form action='{% url "blog-post-detail" post.slug %}' method="post">
            {% csrf_token %}
            <div class='top-div'>
                <div class='top-left'>
                    {{ form.user_name.label_tag }}
                    {% if form.user_name.errors %}
                        {{ form.user_name.errors}}
                    {% endif %}
                    {{ form.user_name }}
                    
                    {{ form.user_email.label_tag }}
                    {% if form.user_email.errors %}
                        {{ form.user_emaile.errors}}
                    {% endif %}
                    {{ form.user_email }}
                </div>
                <div class='top-right'>
                    <button type="submit">Submit Comment</button>
                </div>
            </div>
            <div class="bottom-div">
                {{ form.text.label_tag }}
                {% if form.text.errors %}
                        {{ form.text.errors}}
                    {% endif %}
                {{ form.text }}
            </div>
        </form>
    </div>
{% endblock %}

Here is the css code i added to the post-detail.css file:

In [ ]:
.read-later-form{
    position:absolute;
    bottom:-1rem;
    left:2.2rem;
}
.read-later-form button{
    width:7rem;
    height:2rem;
    border-radius: 8px;
    border:1px solid #c13535;
    background-color:#c13535;
    color:white;
    box-shadow: 0 0 10px rgb(0,0,0,0.6);
    cursor: pointer;
}
.read-later-form button:hover,
.read-later-form button:focus{
    background-color:white;
    color:#c13535;
}

2. We are going to use session to store the read later posts.<br>
3. Add a hidden input to the form, this input is going to have a name an value.
    - Name it 'post_id'
    - Pass {{ post.id }} to its value
4. Navigate to urls.py file and add a new path to it.
    - path('read-later', views.ReadLaterView.as_view(), name='read-later-url')
5. Go to the form you created in the post-detail.html template and use the name set for the new url path to construct a url.
    - {% url 'read-later-url' %}
6. Create a new view class called ReadLaterView and extend it from View.
    - Redefine its post method
    - use request.session.get['stored_posts] to access the stored posts and store them in a variable (stored_posts).
    - Create an if statement and check to make sure that there are stored post avalible.
    - If there is no stored posts in the variable you just created, pass an empty list to it.
    - Now, since we are inthe post method, and this method only gets run when a post request is sent to the server, we need to append the post_id in the request.POST to the variable list we just created.
    - Something you need to make sure here is that you dont add an already existed post to your list.
    - In the end you need to redirect to the '/posts'
Your View class should look like the following block:

In [ ]:
class ReadLaterView(View):
    def post(self, request):
        stored_posts = request.session.get('stored_posts')
        if stored_posts is None:
            stored_posts = []
        post_id = int(request.POST['post_id'])
        if post_id not in stored_posts:
            stored_posts.append(post_id)
        return HttpResponseRedirect(reverse('blog-post-page'))

7. Lets add the page, where we show all the stored posts.<br>
    - Create a new template and its css file (stored-posts.html, stored-posts.css).
    - Lets keep it simple<br>
    
Here is the code for my new template:

In [ ]:
{% extends "base.html" %}
{% load static %}

{% block page_title %}My Stored Posts{% endblock page_title %}

{% block css_files %}
    <link rel="stylesheet" href="{% static "blog/stored-post.css" %}">
{% endblock css_files %}

{% block content %}
    <section>
        <ul>
            <li><a href="">Post Title</a></li>
        </ul>
    </section>
{% endblock content %}

8. Now, we need to define the get method of our new view class.
    - get the stored_posts list using request.session.get('stored_posts)
    - create a context dictionary (empty)
    - check if it is None or has the length of zero
        - If it is the case, create a new key in context, name it posts and pass an empty list to it
        - Also create a key called has_posts and pass False to it
        - If the stored_posts list is not None, fetch the rows from the Post model using Post.objects.filter()
            - use id__in so you can pass more than one umber to it
        - pass the posts to posts key in context and pass True to has_posts
    - return render method and stored-posts.html with the context

here is the code you need to have in get method:

In [ ]:
class ReadLaterView(View):
    def get(self, request):
        stored_posts = request.session.get('stored_posts')
        context = {}
        if stored_posts is None or len(stored_posts)==0:
            context['posts']=[]
            context['has_posts']=False
        else:
            posts = Post.objects.filter(id__in=stored_posts)
            context['posts']=posts
            context['has_posts']=True
        return render(request, 'blog/stored-posts.html', context)

9. add a new link to the main-navigation and construct the url for read-later<br>

In [ ]:
<a href="{% url "read-later-url" %}">Stored Posts</a>

If you save everything and navigate to read-later page, you wont see anything!<br>

The problem is our view<br>

In the post method, we just appened the post id to the stored_post list, but we never wrote it in the session!<br>
To fix this use "request.session['stored_posts']=stored_posts" after appending the post_id to the stored_posts list.<br>

Here is the code for post method:

In [ ]:
def post(self, request):
    stored_posts = request.session.get('stored_posts')
    if stored_posts is None:
        stored_posts = []
    post_id = int(request.POST['post_id'])
    if post_id not in stored_posts:
        stored_posts.append(post_id)
        request.session['stored_posts']=stored_posts
    return HttpResponseRedirect(reverse('blog-post-page'))

Here is the code for stored-posts.html template<br>

In [ ]:
{% extends "base.html" %}
{% load static %}

{% block page_title %}My Stored Posts{% endblock page_title %}

{% block css_files %}
    <link rel="stylesheet" href="{% static "blog/stored-posts.css" %}">
{% endblock css_files %}

{% block content %}
<section id='all-posts'>
    <h2>My Collected Posts</h2>
    <ul>
        {% if has_posts %}
        {% for post in posts %}
            {% include "blog/includes/post.html" %}
        {% endfor %}
        {% else %}
            <li>No Posts are taged as read-later.</li>
        {% endif %}
    </ul>
</section>
{% endblock content %}

Style the template before you move to the next notebook.

In [ ]:
#main-navigation{
    background-color:#338dc4;
}
section{
    margin-top:8rem;
}
#all-posts {
    width:60rem;
    background-color:white;
    padding:2rem;
    margin:7rem auto 2rem auto;
    box-shadow: 1px 1px 12px rgba(0,0,0,0.4);
    text-align: center;
}
#all-posts h2{
    text-align: center;
    font-size:2rem;
    color:#2e2e2e;
    margin:3rem 0;
}
ul{
    list-style: none;
    margin:0;
    padding:0;
    display:grid;
    grid-template-columns:repeat(auto-fill, minmax(17rem, 1fr));  
    gap:1.5rem;
}
li{
    padding:1rem;
    border:2px solid #2e2e2e;
    box-shadow: 0 0 5px rgba(0,0,0,0.6);
    border-radius: 12px;
}
li a{
    text-decoration: none;
    color:black;
}
li:hover,
li:active{
    background-color:#338dc4;
    border-color:transparent;
}
li:hover a,
li:active a{
    color:white;
}
li:hover img,
li:active img{
    border-color:white;
}
.post img{
    width: 7rem;
    height: 7rem;
    border:3px solid #338dc4;
    border-radius: 50%;
}
.post a{
    height: 22rem;
    transform-origin:center;
}

In the next notebook we will add more details.<br>